In [9]:
# Colab cell: install libs we need
%pip install -q fastmcp llama-index llama-index-tools-mcp llama-index-llms-groq groq nest_asyncio aiohttp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 13.6 MB/s eta 0:00:00


In [4]:
# Run this first in Colab. Replace with your key or set via Colab secrets.
import os
from dotenv import load_dotenv

load_dotenv()

# Option A: paste (quick, not recommended for sharing)
os.environ["GROQ_API_KEY"] = "api-key"

# Option B (recommended): use Colab UI -> Secrets and then set here:
# from google.colab import auth
# (or use the "secrets" UI to set an env var)
print("GROQ_API_KEY set (make sure you replaced the placeholder).")


GROQ_API_KEY set (make sure you replaced the placeholder).


In [23]:
import sqlite3, threading, time
from fastmcp import FastMCP

conn = sqlite3.connect("demo.db")
conn.execute("""
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    info TEXT
)""")

conn.commit()
conn.close()

mcp = FastMCP(name = "colab sqlite MCP", port = 9090)

def get_conn():
  return sqlite3.connect("demo.db", check_same_thread = False)

@mcp.tool()
def add_data(name: str, info: str):
  """Insert (name, info) into the database"""
  conn = get_conn()
  cur = conn.cursor()
  cur.execute("INSERT INTO people (name, info) VALUES (?, ?)", (name, info))
  conn.commit()
  conn.close()
  return "OK"

@mcp.tool()
def read_data() -> str:
  """Return a text dump of the people table."""
  conn = get_conn()
  cur = conn.cursor()
  rows = cur.execute("SELECT id, name, info FROM people")
  conn.close()
  return "\n".join([f"{r[0]} | {r[1]} | {r[2]}" for r in rows])

def run_server():
  mcp.run(transport = "sse", port = 9090)

thread = threading.Thread(target = run_server, daemon = True)
thread.start()
time.sleep(1)
print("MCP server should be up at http://127.0.0.1:9090/sse")


╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     colab sqlite MCP                                    │
│    📦 Transport:       SSE                                                 │
│    🔗 Server URL:      http://127.0.0.1:9090/sse/                          │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.11.2                                              │
│    🤝 MCP version:     1.12.4                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

[08/11/25 05:19:40] INFO     Starting MCP server 'colab sqlite MCP' with transport 'sse' on          ]8;id=887209;file:///usr/local/lib/python3.11/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=936241;file:///usr/local/lib/python3.11/dist-packages/fastmcp/server/server.py#1522\1522]8;;\
                             http://127.0.0.1:9090/sse/                                                            

INFO:     Started server process [990]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 9090): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


MCP server should be up at http://127.0.0.1:9090/sse


In [24]:
# test the MCP server by listing and calling tools
import asyncio, nest_asyncio
from llama_index.tools.mcp import BasicMCPClient

nest_asyncio.apply()  # allow nested asyncio in Colab

async def demo():
    client = BasicMCPClient("http://127.0.0.1:9090/sse")
    tools = await client.list_tools()
    print("TOOLS AVAILABLE:")
    for name, desc in tools:
          print("-", name, "|", desc)


    # add a row
    res = await client.call_tool("add_data", {"name": "Hari", "info": "ML engineer wannabe"})
    print("add_data result:", res)

    # read rows
    res2 = await client.call_tool("read_data", {})
    print("read_data result:\n", res2)

asyncio.run(demo())


INFO:     127.0.0.1:47318 - "GET /sse HTTP/1.1" 307 Temporary Redirect


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='functools' coro=<AsyncIOBackend.run.<locals>.wrapper() done, defined at /usr/local/lib/python3.11/dist-packages/anyio/_backends/_asyncio.py:2297> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('127.0.0.1', 9090): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipyth

INFO:     127.0.0.1:47318 - "GET /sse/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:47334 - "POST /messages/?session_id=7fd36c6ab1a74a02ba28c7b8defb5fb7 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:47334 - "POST /messages/?session_id=7fd36c6ab1a74a02ba28c7b8defb5fb7 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:47334 - "POST /messages/?session_id=7fd36c6ab1a74a02ba28c7b8defb5fb7 HTTP/1.1" 202 Accepted
TOOLS AVAILABLE:
- meta | None
- nextCursor | None
- tools | [Tool(name='add_data', title=None, description='Insert (name, info) into people table.', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 'info': {'title': 'Info', 'type': 'string'}}, 'required': ['name', 'info'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta={'_fastmcp': {'tags': []}}), Tool(name='read_data', title=None, description='Return a t

In [26]:
import os
from llama_index.llms.groq import Groq

llm = Groq(model = "llama3-70b-8192", api_key = os.environ.get("GROQ_API_KEY"))

response = llm.complete("Tell me about LFC in one line.")
response

CompletionResponse(text='Liverpool Football Club (LFC) is a professional football club based in Liverpool, England, that has won numerous domestic and international titles, including six European Cups, 19 League titles, and seven FA Cups, with a rich history and a passionate fan base known for their iconic anthem "You\'ll Never Walk Alone".', additional_kwargs={'prompt_tokens': 19, 'completion_tokens': 64, 'total_tokens': 83}, raw=ChatCompletion(id='chatcmpl-4e6ab2ad-6494-4124-a2bf-43df7a1a6812', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Liverpool Football Club (LFC) is a professional football club based in Liverpool, England, that has won numerous domestic and international titles, including six European Cups, 19 League titles, and seven FA Cups, with a rich history and a passionate fan base known for their iconic anthem "You\'ll Never Walk Alone".', refusal=None, role='assistant', annotations=None, audio=None, function_call=N

In [31]:
import asyncio, nest_asyncio
from llama_index.tools.mcp import aget_tools_from_mcp_url

nest_asyncio.apply()

async def fetch_tools():
  tools = await aget_tools_from_mcp_url("http://127.0.0.1:9090/sse")
  return tools

tools = asyncio.run(fetch_tools())
print("Converted tools for agent")
for t in tools:
    print("-", t.metadata.name, ":", t.metadata.description)


INFO:     127.0.0.1:48462 - "GET /sse HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:48462 - "GET /sse/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:48474 - "POST /messages/?session_id=832b259c0e7e4090b658916370f8626f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:48474 - "POST /messages/?session_id=832b259c0e7e4090b658916370f8626f HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:48474 - "POST /messages/?session_id=832b259c0e7e4090b658916370f8626f HTTP/1.1" 202 Accepted
Converted tools for agent
- add_data : Insert (name, info) into people table.
- read_data : Return a text dump of the people table.


In [33]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.tools import FunctionTool
agent = ReActAgent(

      name = "sqlite_agent",
      description="Agent that can add and read people in SQLite via MCP tools.",
      system_prompt="You are a helpful assistant. Use available tools to read/add people in the database.",
      llm = llm,
      tools = tools

)

resp =await agent.run("add a person named alice who loves climbing. then show me all people.")
print("Agent response:\n", resp)

/usr/lib/python3.11/inspect.py:573: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/usr/lib/python3.11/inspect.py:573: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  value = getter(object, key)
/usr/lib/python3.11/inspect.py:573: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  value = getter(object, key)
/usr/lib/python3.11/inspect.py:573: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instanc

INFO:     127.0.0.1:59516 - "GET /sse HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:59516 - "GET /sse/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:59524 - "POST /messages/?session_id=42be08a1118c48639cb2c8c8eaf672e0 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59524 - "POST /messages/?session_id=42be08a1118c48639cb2c8c8eaf672e0 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59524 - "POST /messages/?session_id=42be08a1118c48639cb2c8c8eaf672e0 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59524 - "POST /messages/?session_id=42be08a1118c48639cb2c8c8eaf672e0 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59538 - "GET /sse HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:59538 - "GET /sse/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:59544 - "POST /messages/?session_id=d5a1dec2fa544d66a425647479f6ce7a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59544 - "POST /messages/?session_id=d5a1dec2fa544d66a425647479f6ce7a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59544 - "POST /messages/?session_id=d5a1dec2fa544d66a4